In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
import torch.nn.functional as F

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:

trainset = torchvision.datasets.CIFAR100(root='./data', train = True, transform = data_transforms, download=True)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, transform = data_transforms,
download=True)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=True)

100%|██████████| 169001437/169001437 [00:12<00:00, 13350670.37it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3,1,1, padding_mode='replicate')
        self.conv2 = nn.Conv2d(32, 32, 3,1,1, padding_mode='replicate')
        self.conv3 = nn.Conv2d(32, 64, 3,1,1, padding_mode='replicate')
        self.fc1 = nn.Linear(28*28*64, 5000)
        self.fc2 = nn.Linear(5000, 100)
        self.pool = nn.MaxPool2d(2, 2)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 28*28*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Model()

In [ ]:
num_epochs = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:

train = int(input("Enter 1 to train:"))
if train == 1:
  model.train()
  for epoch in range(num_epochs):

      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          inputs, labels = data
          inputs, labels = inputs.cuda(), labels.cuda()


          optimizer.zero_grad()


          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()


          running_loss += loss.item()
          if i % 2000 == 1999:    # print mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
              running_loss = 0.0


  torch.save(model.state_dict(), 'best_model.pth')
  print('Finished Training')
else:
  model.eval()

Enter 1 to train otherwise 0:1
[1,  2000] loss: 3.498
[2,  2000] loss: 2.521
[3,  2000] loss: 2.032
[4,  2000] loss: 1.529
[5,  2000] loss: 1.024
[6,  2000] loss: 0.653
[7,  2000] loss: 0.429
[8,  2000] loss: 0.296
[9,  2000] loss: 0.223
[10,  2000] loss: 0.180
[11,  2000] loss: 0.150
[12,  2000] loss: 0.121
[13,  2000] loss: 0.099
[14,  2000] loss: 0.092
[15,  2000] loss: 0.087
[16,  2000] loss: 0.072
[17,  2000] loss: 0.066
[18,  2000] loss: 0.062
[19,  2000] loss: 0.063
[20,  2000] loss: 0.059
[21,  2000] loss: 0.061
[22,  2000] loss: 0.051
[23,  2000] loss: 0.051
[24,  2000] loss: 0.042
[25,  2000] loss: 0.044
[26,  2000] loss: 0.045
[27,  2000] loss: 0.041
[28,  2000] loss: 0.040
[29,  2000] loss: 0.034
[30,  2000] loss: 0.036
Finished Training


In [ ]:
true_pos = 0
total_detect = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_detect += labels.size(0)
        true_pos += (predicted == labels).sum().item()

print('Accuracy : %d %%' % (
    100 * true_pos / total_detect))

Accuracy of the network on the 10000 test images: 37 %
